In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

turnstile_file = '/Users/busola.oladapo/Documents/Github/Project_Benson/turnstile_181229.txt'
station_file = '/Users/busola.oladapo/Documents/Github/Project_Benson/Stations.csv'
income_file = '/Users/busola.oladapo/Documents/Github/Project_Benson/Income by Zip code.xlsx'

project_name = 'Project Benson'

In [2]:
# load turnstile data
data_turnstile = pd.read_csv(turnstile_file)
print(f'Turnstile data: {data_turnstile.shape[0]} rows')

data_turnstile = data_turnstile[['STATION']]
data_turnstile.drop_duplicates(inplace=True)
print(f'Turnstile data after deduplicating: {data_turnstile.shape[0]} rows')
data_turnstile.head()


Turnstile data: 201174 rows
Turnstile data after deduplicating: 376 rows


,STATION
0,59 ST
504,5 AV/59 ST
1100,57 ST-7 AV
1690,49 ST
2620,TIMES SQ-42 ST


In [3]:
# load station data
data_station = pd.read_csv(station_file)
print(f'Station data: {data_station.shape[0]} rows')

data_station['STATION'] = data_station['Stop Name'].str.upper()
data_station = data_station[['STATION', 'GTFS Latitude', 'GTFS Longitude']]
data_station.head()

Station data: 496 rows


,STATION,GTFS Latitude,GTFS Longitude
0,ASTORIA - DITMARS BLVD,40.775036,-73.912034
1,ASTORIA BLVD,40.770258,-73.917843
2,30 AV,40.766779,-73.921479
3,BROADWAY,40.761820,-73.925508
4,36 AV,40.756804,-73.929575


In [4]:
station_locations = pd.merge(data_turnstile, data_station, on='STATION', how='left')    # TODO: some stations have multiple longitude/latitudes, investigate
print(f'Station Location data: {station_locations.shape[0]} rows')
station_locations.head()


Station Location data: 474 rows


,STATION,GTFS Latitude,GTFS Longitude
0,59 ST,40.641362,-74.017881
1,59 ST,40.762526,-73.967967
2,5 AV/59 ST,40.764811,-73.973347
3,57 ST-7 AV,NaN,NaN
4,49 ST,40.759901,-73.984139


In [5]:
def create_coordinates(row):
    return str(row['GTFS Latitude']) + ', ' + str(row['GTFS Longitude'])

station_locations.dropna(inplace=True)  # TODO, may need to get this information from other sources
# create a coordinates column from the latitude and longitude fields
station_locations['coordinates'] = station_locations.apply(create_coordinates, axis=1)
station_locations.head()

,STATION,GTFS Latitude,GTFS Longitude,coordinates
0,59 ST,40.641362,-74.017881,"40.641362, -74.017881"
1,59 ST,40.762526,-73.967967,"40.762526, -73.967967"
2,5 AV/59 ST,40.764811,-73.973347,"40.764811, -73.973347"
4,49 ST,40.759901,-73.984139,"40.759901, -73.984139"
7,28 ST,40.745494,-73.988691,"40.745494, -73.988691"


In [6]:
geolocator = Nominatim(user_agent=project_name)

def get_zipcode(loc):
    '''
    Translates latitude/longitude coordinates to zipcode
    '''
    try:
        location = geolocator.reverse(loc)
        return (location.raw)['address']['postcode']
    except GeocoderTimedOut:
        return get_zipcode(loc)    # try again if timeout error
    except Exception:
        return np.nan              # null if zipcode doesn't exist in result
    

station_locations['zipcode'] = station_locations['coordinates'].apply(get_zipcode)
station_locations.head()

,STATION,GTFS Latitude,GTFS Longitude,coordinates,zipcode
0,59 ST,40.641362,-74.017881,"40.641362, -74.017881",11220
1,59 ST,40.762526,-73.967967,"40.762526, -73.967967",10037
2,5 AV/59 ST,40.764811,-73.973347,"40.764811, -73.973347",10153
4,49 ST,40.759901,-73.984139,"40.759901, -73.984139",10019
7,28 ST,40.745494,-73.988691,"40.745494, -73.988691",10001


In [7]:
station_locations[station_locations['zipcode'].isnull()]  # TODO, decide on approach for nulls, 9 on initial count. 2 aftter addressing timeout exception


,STATION,GTFS Latitude,GTFS Longitude,coordinates,zipcode
106,50 ST,40.761728,-73.983849,"40.761728000000005, -73.98384899999999",NaN
206,72 ST,40.778453,-73.981970,"40.778453000000006, -73.98196999999999",NaN


In [30]:
data_income = pd.read_excel(income_file)
data_income = data_income[['Zip code', 'Size of adjusted gross income', 'Number of single returns', 'Number of joint returns', 'Number of head of household returns']]
data_income = data_income.rename(columns={'Zip code': 'zipcode'
                                          ,'Size of adjusted gross income': 'agi'
                                          , 'Number of single returns': 'single'
                                          , 'Number of joint returns': 'joint'
                                          , 'Number of head of household returns': 'hoh'})
data_income.head(15)
# data_income['Size of adjusted gross income'].unique()


,zipcode,agi,single,joint,hoh
0,0,Total,4848730.0,2939200.0,1480010.0
1,0,"$1 under $25,000",2354910.0,445530.0,595140.0
2,0,"$25,000 under $50,000",1161980.0,441060.0,474430.0
3,0,"$50,000 under $75,000",644420.0,398720.0,216490.0
4,0,"$75,000 under $100,000",307400.0,392440.0,99910.0
5,0,"$100,000 under $200,000",293860.0,841230.0,82300.0
6,0,"$200,000 or more",86160.0,420220.0,11740.0
7,NaN,NaN,NaN,NaN,NaN
8,10001,NaN,10450.0,2540.0,1120.0
9,10001,"$1 under $25,000",2890.0,350.0,400.0


In [35]:
data_income_100_200 = data_income[data_income['agi'] == '$100,000 under $200,000']
data_income_over_200 = data_income[data_income['agi'] == '$200,000 or more']

income_zip = data_income_100_200[['zipcode', 'single']].rename(columns={'single': 'num_returns'})
data_income_100_200_hoh = data_income_100_200[['zipcode', 'hoh']].rename(columns={'hoh': 'num_returns'})
data_income_over_200['total'] = data_income_over_200['single'] + data_income_over_200['joint'] + data_income_over_200['hoh']
data_income_over_200_all = data_income_over_200[['zipcode', 'total']].rename(columns={'total': 'num_returns'})

income_zip = income_zip.append(data_income_100_200_hoh, ignore_index=True).append(data_income_over_200_all, ignore_index=True)
income_zip['zipcode'] = income_zip['zipcode'].astype('str')
income_zip = income_zip.groupby('zipcode').sum().reset_index()

income_zip.shape
income_zip.head(15)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,zipcode,num_returns
0,0,894280.0
1,10001,4220.0
2,10002,4040.0
3,10003,11130.0
4,10004,1200.0
5,10005,2670.0
6,10006,1020.0
7,10007,1970.0
8,10009,5590.0
9,10010,6960.0


In [37]:

# income_zip[income_zip['zipcode'] == '11220']

# station_locations.head()
station_income = pd.merge(station_locations, income_zip, on='zipcode', how='left')
station_income.head(20)

,STATION,GTFS Latitude,GTFS Longitude,coordinates,zipcode,num_returns
0,59 ST,40.641362,-74.017881,"40.641362, -74.017881",11220,1030.0
1,59 ST,40.762526,-73.967967,"40.762526, -73.967967",10037,790.0
2,5 AV/59 ST,40.764811,-73.973347,"40.764811, -73.973347",10153,NaN
3,49 ST,40.759901,-73.984139,"40.759901, -73.984139",10019,9030.0
4,28 ST,40.745494,-73.988691,"40.745494, -73.988691",10001,4220.0
5,28 ST,40.747215,-73.993365,"40.747215000000004, -73.993365",10001,4220.0
6,28 ST,40.743070,-73.984264,"40.74307, -73.984264",10016,11420.0
7,23 ST,40.741303,-73.989344,"40.741303, -73.98934399999999",10010,6960.0
8,23 ST,40.745906,-73.998041,"40.745906, -73.998041",10011,12800.0
9,23 ST,40.742878,-73.992821,"40.742878000000005, -73.99282099999999",10019,9030.0
